In [35]:
import os
import csv
from collections import defaultdict
import json

# To finetune

In [36]:
DATASET_VAR = 'augm_final_1'        # augm_final_1, base_final_1, orig
DATASET_NAME = 'dataset'
RESULTS_PATH = f'/home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/{DATASET_VAR}'
VALID_TABLE = f'{RESULTS_PATH}/valid/BirdNET_SelectionTable.txt'
TRAIN_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/train'
VALID_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/valid'
TEST_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/test'
TEST_TABLE = f'{RESULTS_PATH}/test/BirdNET_SelectionTable.txt'
FINAL_RESULTS_PATH = '/home/giacomoschiavo/finetuning-BirdNET/models/results'

In [37]:
print(f'python -m birdnet_analyzer.train --i {TRAIN_PATH} --o {RESULTS_PATH}/{DATASET_VAR}.tflite --batch_size 64 --threads 16 --val_split 0.01 --epochs 150 --mixup --cache_mode save --cache_file {RESULTS_PATH}/{DATASET_VAR}.npz')

python -m birdnet_analyzer.train --i /home/giacomoschiavo/segments/dataset/train --o /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/augm_final_1/augm_final_1.tflite --batch_size 64 --threads 16 --val_split 0.01 --epochs 150 --mixup --cache_mode save --cache_file /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/augm_final_1/augm_final_1.npz


# To analize

In [38]:
MIN_CONF = '0.05'
SENS = '1'

In [39]:
print(f'python -m birdnet_analyzer.analyze --i {VALID_PATH} --o {RESULTS_PATH}/valid --slist /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/Labels.txt --threads 16 --combine_results --min_conf {MIN_CONF} --classifier {RESULTS_PATH}/{DATASET_VAR}.tflite --sensitivity {SENS}')

python -m birdnet_analyzer.analyze --i /home/giacomoschiavo/segments/dataset/valid --o /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/augm_final_1/valid --slist /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/Labels.txt --threads 16 --combine_results --min_conf 0.05 --classifier /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/augm_final_1/augm_final_1.tflite --sensitivity 1


In [40]:
print(f'python -m birdnet_analyzer.analyze --i {TEST_PATH} --o {RESULTS_PATH}/test --slist /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/Labels.txt --threads 16 --combine_results --min_conf {MIN_CONF} --classifier {RESULTS_PATH}/{DATASET_VAR}.tflite --sensitivity {SENS}')

python -m birdnet_analyzer.analyze --i /home/giacomoschiavo/segments/dataset/test --o /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/augm_final_1/test --slist /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/Labels.txt --threads 16 --combine_results --min_conf 0.05 --classifier /home/giacomoschiavo/finetuning-BirdNET/models/BirdNET_tuned/augm_final_1/augm_final_1.tflite --sensitivity 1


# Analysis

In [41]:
with open(f"./utils/{DATASET_NAME}/dataset_config_custom.json") as f:
    dataset_config = json.load(f)
class_names = list(dataset_config['mappings'].keys())

In [42]:
with open('/home/giacomoschiavo/finetuning-BirdNET/utils/species_dict_map.json') as f:
    species_dict = json.load(f)
inv_species_dict = {value: key for key, value in species_dict.items()}

In [43]:
def get_conf_scores():
    conf_scores = {}

    #  {'Aeroplane': [(np.float32(0.0), False),
    #               (np.float32(1.3937646e-30), False),
    #               (np.float32(1.0654355e-25), False),
    #               (np.float32(0.0), False),

    with open(VALID_TABLE, 'r') as f:
        reader = csv.DictReader(f, delimiter='\t')  # Usa TAB come separatore
        for row in reader:
            file_path = row['Begin Path']
            pred_species_name = row['Common Name']
            if row['Common Name'] in inv_species_dict:
                pred_species_name = '_'.join([inv_species_dict[row['Common Name']], row['Common Name']])
            true_species_name = file_path.split('/')[-2]
            confidence = float(row['Confidence'])
            if pred_species_name not in conf_scores:
                conf_scores[pred_species_name] = []
            is_correct = pred_species_name == true_species_name
            conf_scores[pred_species_name].append((confidence, is_correct))
    return conf_scores
            

In [44]:
conf_scores = get_conf_scores()

In [45]:
from sklearn.metrics import f1_score
import numpy as np 

DEFAULT_THRESH = 0.15

def compute_best_thresholds(conf_scores, num_thresholds=200, min_thresh=0.01, max_thresh=0.95):
    thresholds = {}

    for species, values in conf_scores.items():
        probs, truths = zip(*values)
        probs = np.array(probs)
        truths = np.array(truths).astype(int)

        best_thresh = DEFAULT_THRESH
        best_f1 = 0.0

        for thresh in np.linspace(min_thresh, max_thresh, num_thresholds):
            preds = (probs >= thresh).astype(int)
            f1 = f1_score(truths, preds, zero_division=0)
            if f1 > best_f1:
                best_f1 = f1
                best_thresh = thresh

        thresholds[species] = best_thresh
        print(f"📊 {species} -> {best_thresh:.3f}, F1-score: {best_f1:.3f}")

    return thresholds



In [46]:
best_thresholds = compute_best_thresholds(conf_scores)

📊 Phylloscopus collybita_Common Chiffchaff -> 0.053, F1-score: 0.222
📊 Fringilla coelebs_Common Chaffinch -> 0.208, F1-score: 0.446
📊 Sylvia atricapilla_Eurasian Blackcap -> 0.150, F1-score: 0.000
📊 Troglodytes troglodytes_Eurasian Wren -> 0.119, F1-score: 0.431
📊 Erithacus rubecula_European Robin -> 0.071, F1-score: 0.222
📊 Turdus merula_Eurasian Blackbird -> 0.473, F1-score: 0.245
📊 Lophophanes cristatus_Crested Tit -> 0.147, F1-score: 0.182
📊 Loxia curvirostra_Common Crossbill -> 0.312, F1-score: 0.416
📊 Anthus trivialis_Tree Pipit -> 0.251, F1-score: 0.242
📊 Parus major_Great Tit -> 0.100, F1-score: 0.018
📊 Regulus ignicapilla_Common Firecrest -> 0.071, F1-score: 0.248
📊 Periparus ater_Coal Tit -> 0.591, F1-score: 0.433
📊 Certhia familiaris_Eurasian Treecreeper -> 0.799, F1-score: 0.762
📊 None -> 0.062, F1-score: 0.529
📊 Wind -> 0.086, F1-score: 0.270
📊 Coccothraustes coccothraustes_Hawfinch -> 0.076, F1-score: 0.514
📊 Regulus regulus_Goldcrest -> 0.090, F1-score: 0.941
📊 Vegetatio

In [47]:
with open(f'/home/giacomoschiavo/finetuning-BirdNET/utils/{DATASET_NAME}/true_segments_test.json') as f:
    true_segments_test = json.load(f)

In [48]:
with open(f'/home/giacomoschiavo/finetuning-BirdNET/utils/{DATASET_NAME}/true_segments_train.json') as f:
    true_segments_full = json.load(f)

In [49]:
true_segments_full.update(true_segments_test)

In [50]:
from sklearn.preprocessing import MultiLabelBinarizer

test_species_list = os.listdir(TEST_PATH)
# test_species_list = [species for species in test_species_list if len(species.split('_')) > 1]
mlb = MultiLabelBinarizer()
mlb.fit([test_species_list])
# if DATASET_VAR == "orig":
#     class_names = [species for species in test_species_list if len(species.split("_")) > 1]
#     mlb.fit([class_names])

len(mlb.classes_)

20

In [51]:
pred_segments_proba = {}
with open(TEST_TABLE, 'r') as f:
    reader = csv.DictReader(f, delimiter='\t')  # Usa TAB come separatore
    for row in reader:
        file_path = row['Begin Path']
        audio_name = os.path.basename(file_path)    # in test non sono presenti WABAD o augm
        only_audio_name = "_".join(audio_name.split("_")[:2]) + ".WAV"
        segm = "_".join(audio_name.split(".")[0].split("_")[-2:])
        pred_species_name = row['Common Name']
        if row['Common Name'] in inv_species_dict:
            pred_species_name = '_'.join([inv_species_dict[row['Common Name']], row['Common Name']])
        # if len(file_path.split('/')[-2].split('_')) == 1:       # skip if None, Pecking, etc. 
        #     continue
        confidence = float(row['Confidence'])
        pred_segments_proba.setdefault(only_audio_name, {})
        best_thresholds.setdefault(pred_species_name, DEFAULT_THRESH)
        if confidence >= best_thresholds[pred_species_name]:
            pred_segments_proba[only_audio_name].setdefault(segm, {})
            if "None" in pred_segments_proba[only_audio_name][segm]:
                continue
            if pred_species_name == "None":
                pred_segments_proba[only_audio_name][segm] = {"None": confidence}
            else:
                pred_segments_proba[only_audio_name][segm].update({pred_species_name: confidence})

In [52]:
true_segments = defaultdict(dict)
for species in os.listdir(TEST_PATH):
    if species not in test_species_list:
        print(species, 'ignored')
        continue
    for audio in os.listdir(os.path.join(TEST_PATH, species)):
        audio = audio.split('.')[0]
        date, time, segm1, segm2 = audio.split('_')
        audio_name = '_'.join([date, time]) + '.WAV'
        segm = '_'.join([segm1, segm2])
        if segm not in true_segments[audio_name]:
            true_segments[audio_name][segm] = []
        true_segments[audio_name][segm].extend([species])

In [53]:
for audio in true_segments.keys():
    pred_segments_proba.setdefault(audio, {})
    if audio in pred_segments_proba:
        for segm in true_segments[audio].keys():
            pred_segments_proba[audio].setdefault(segm, {})

In [54]:
# extract recognized labels
pred_segments = {}
pred_proba = {}

for audio, segments in pred_segments_proba.items():
    pred_segments.setdefault(audio, {})
    pred_proba.setdefault(audio, {})
    for segm, labels in segments.items():
        pred_segments[audio].setdefault(segm, {})
        pred_segments[audio][segm] = list(labels.keys())
        pred_proba[audio].setdefault(segm, {})
        pred_proba[audio][segm] = list(labels.values())

In [55]:
def segment_key_to_float(k):
    return float(k.replace("_", "."))

def get_conf_dict(pred_segments):
    for audio in pred_segments.keys():
        pred_segments[audio] = dict(sorted(pred_segments[audio].items(), key=lambda x: segment_key_to_float(x[0]))) 
    pred_segments = dict(sorted(pred_segments.items(), key=lambda x: x[0])) 
    return pred_segments

In [56]:
pred_segments_proba = get_conf_dict(pred_segments_proba)

In [57]:
with open(f"{RESULTS_PATH}/test_pred_segments.json", 'w') as f:
    json.dump(pred_segments_proba, f, indent=4)

In [58]:
y_pred = []
y_true = []
y_pred_proba = []

for audio in pred_segments:
    for segment in sorted(pred_segments[audio].keys()):
        true_labels = true_segments[audio].get(segment, [])
        pred_labels = pred_segments[audio].get(segment, [])
        proba_values = pred_proba[audio].get(segment, [])

        y_true_vec = mlb.transform([true_labels])[0]  # 1D array
        y_pred_vec = mlb.transform([pred_labels])[0]  # 1D array

        proba_vec = np.zeros(len(mlb.classes_))
        for label, score in zip(pred_labels, proba_values):
            if label in mlb.classes_:
                idx = list(mlb.classes_).index(label)
                proba_vec[idx] = score

        y_true.append(y_true_vec)
        y_pred.append(y_pred_vec)
        y_pred_proba.append(proba_vec)

y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_pred_proba = np.array(y_pred_proba)


In [59]:
from sklearn.metrics import classification_report
import pandas as pd
report = classification_report(y_true, y_pred, target_names=mlb.classes_, zero_division=0, output_dict=True)

report_df = pd.DataFrame(report).T
report_df

,precision,recall,f1-score,support
Aeroplane,0.000000,0.000000,0.000000,22.0
Anthus trivialis_Tree Pipit,0.523810,0.388235,0.445946,170.0
Certhia familiaris_Eurasian Treecreeper,0.270270,0.101010,0.147059,99.0
Coccothraustes coccothraustes_Hawfinch,0.772727,0.195402,0.311927,87.0
Erithacus rubecula_European Robin,0.466286,0.733813,0.570231,556.0
Fringilla coelebs_Common Chaffinch,0.512315,0.686675,0.586817,1666.0
Lophophanes cristatus_Crested Tit,0.034091,0.130435,0.054054,23.0
Loxia curvirostra_Common Crossbill,0.458333,0.239130,0.314286,46.0
Muscicapa striata_Spotted Flycatcher,0.000000,0.000000,0.000000,173.0
None,0.821805,0.915866,0.866290,4683.0


In [60]:
with open(f'{RESULTS_PATH}/classification_report.json', 'w') as f:
    json.dump(report, f)

In [61]:
np.savez(f'{FINAL_RESULTS_PATH}/results_b{DATASET_VAR}.npz', y_true=y_true, y_pred=y_pred, y_pred_proba=y_pred_proba, class_names=mlb.classes_)